In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
import config
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from matplotlib import pyplot
from keras.callbacks import History 
import h5py

In [14]:

def read_files():
    tfidf_df = pd.read_csv(config.datasets_dir + config.tfidf_file_name)
    #print(list(tfidf_df.columns)[:30])
    clean_df = pd.read_csv(config.datasets_dir + config.clean_csv_name)
    df = tfidf_df
    df['ASSET_CLASS'] = clean_df['ASSET_CLASS']
    return df

In [15]:
def trainTestSplit(df,n):
    random.seed(123)
    df1 = df['ASSET_CLASS'].value_counts().rename_axis('Assets').reset_index(name = 'counts')
    df_new = df1[df1['counts']>=n] # Train Test split 75% - train
    assets = list(df_new['Assets'])
    dffiltered = df[df['ASSET_CLASS'].isin(assets)]
    dffiltered['ASSET_CLASS_CODES'] = pd.Categorical(dffiltered['ASSET_CLASS'])
    dffiltered['ASSET_CLASS_CODES'] = dffiltered['ASSET_CLASS_CODES'].cat.codes
    x = dffiltered.drop(columns = ['ASSET_CLASS','ASSET_CLASS_CODES','important_words'])
    xcols = list(x.columns)
    y = dffiltered['ASSET_CLASS_CODES']
    X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size = 0.20, stratify = y)
    print(' Number of Assets ' + str(len(set(list(dffiltered['ASSET_CLASS'])))))
    #dict_codes = pd.Series(df.ASSET_CLASS.values, index = df.ASSET_CLASS_CODES).to_dict()
    return X_train, X_test, Y_train, Y_test

In [20]:
def scores(y_pred, Y_test):
    print('Hiiii')
    print('Accuracy:   '+str(accuracy_score(y_pred, Y_test)))
    print('Precision Macro:   '+ str(precision_score(y_pred, Y_test,average = 'macro')))
    print('Recall Macro:     '+str(recall_score(y_pred, Y_test, average = 'macro')))
    print('F1 Score Macro:     '+str(f1_score(y_pred, Y_test, average = 'macro')))
    print('\n')

In [35]:

def MLP(X_train, X_test, Y_train, Y_test):
    print(X_train.shape)
    print(X_train.shape[1])

    model = Sequential()
    model.add(Dense(1000, input_dim = X_train.shape[1],activation = 'softmax'))
    model.add(Dense(750, activation = 'softmax'))
    model.add(Dense(500, activation = 'softmax'))
    model.add(Dense(204, activation = 'softmax'))
    # compile keras
    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.summary()
    # Early stopping
    #es = EarlyStopping(monitor = 'val_loss', mode=1, verbose=3.1)
    #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    # Fit the keras
    history = History()
    #mc = ModelCheckpoint(config.mlp_model_data1, monitor='val_loss', verbose=1, save_best_only=True)
    #callbacks_list = [checkpoint]
    history = model.fit(X_train, Y_train, epochs = 10, batch_size = 32, validation_split = 0.1,verbose = 1,callbacks=[history])
    # Model Evaluation
    _, train_acc = model.evaluate(X_train, Y_train, verbose =2)
    _, test_acc = model.evaluate(X_test, Y_test, verbose=2)
    # predicted classes
    y_pred = model.predict_classes(X_test)
    scores(y_pred, Y_test)
    print('Train Accuracy  '+str(train_acc))
    print('Test Accuracy   '+str(test_acc))
    pyplot.plot(history.history['loss'], label = 'train')
    pyplot.plot(history.history['val_loss'], label = 'Validation')
    pyplot.xlabel('Epochs')
    pyplot.ylabel('Loss')
    pyplot.legend()
    pyplot.show()
    
    # save the best Model
    # save best Model
    

In [36]:
def main():
    df = read_files()
    n = 100
    X_train, X_test, Y_train, Y_test = trainTestSplit(df,n)
    print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
    MLP(X_train, X_test, Y_train, Y_test)
    #print(len(Y_train.unique()))

In [37]:
main()

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


 Number of Assets 204
(72477, 169) (18120, 169) (72477,) (18120,)
(72477, 169)
169
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 1000)              170000    
_________________________________________________________________
dense_21 (Dense)             (None, 750)               750750    
_________________________________________________________________
dense_22 (Dense)             (None, 500)               375500    
_________________________________________________________________
dense_23 (Dense)             (None, 204)               102204    
Total params: 1,398,454
Trainable params: 1,398,454
Non-trainable params: 0
_________________________________________________________________


AttributeError: module 'config' has no attribute 'mlp_model_data1'

In [39]:
x = [1,2,3,4,5]
y = x[2:]

In [40]:
y

[3, 4, 5]